In [144]:

from dash import Dash, html, dash_table, dcc, callback, Output, Input, State
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Load the data
df = pd.read_csv('gyroscope_data_cleaned.csv')

# Print column names to verify
print("DataFrame columns:", df.columns)

app = Dash()

app.layout = html.Div([
    html.Div(children='LSM6DS3 Readings'),
    html.Hr(),
    dcc.Dropdown(
        options=[
            {'label': 'X-axis (xx)', 'value': 'X-axis (xx)'},
            {'label': 'Y-axis (yy)', 'value': 'Y-axis (yy)'},
            {'label': 'Z-axis (zz)', 'value': 'Z-axis (zz)'},
            {'label': 'All (x, y, z)', 'value': 'All'}
        ],
        value='X-axis (xx)',
        id='data-column'
    ),
    dcc.Dropdown(
        options=[
            {'label': 'Line Chart', 'value': 'line'},
            {'label': 'Scatter Plot', 'value': 'scatter'},
            {'label': 'Distribution Plot', 'value': 'distribution'}
        ],
        value='line',
        id='graph-type'
    ),
    html.Div([
        html.Label('Number of samples to display:'),
        dcc.Input(id='num-samples', type='number', value=100, min=1)
    ]),
    html.Button('Previous', id='prev-btn', n_clicks=0),
    html.Button('Next', id='next-btn', n_clicks=0),
    dash_table.DataTable(id='summary-table', columns=[
        {"name": "Metric", "id": "metric"},
        {"name": "Value", "id": "value"}
    ], style_table={'overflowX': 'auto'}),
    dcc.Graph(figure={}, id='graph')
])

@callback(
    [Output('graph', 'figure'),
     Output('summary-table', 'data')],
    [Input('data-column', 'value'),
     Input('graph-type', 'value'),
     Input('num-samples', 'value'),
     Input('prev-btn', 'n_clicks'),
     Input('next-btn', 'n_clicks')],
    [State('graph', 'figure')]
)
def update_graph(col_chosen, graph_type, num_samples, prev_clicks, next_clicks, current_figure):
    # Get the total number of rows in the dataframe
    total_samples = len(df)
    
    # Get the current start and end indices for the displayed data
    start_idx = 0
    end_idx = min(num_samples, total_samples)
    
    if 'layout' in current_figure:
        # Determine current start and end index from the existing figure's layout
        try:
            start_idx = current_figure['layout']['annotations'][0]['text'].split('-')[0]
            end_idx = current_figure['layout']['annotations'][0]['text'].split('-')[1]
            start_idx = int(start_idx)
            end_idx = int(end_idx)
        except (KeyError, IndexError):
            start_idx = 0
            end_idx = min(num_samples, total_samples)

    # Update indices based on button clicks
    if prev_clicks > 0:
        start_idx = max(start_idx - num_samples, 0)
        end_idx = start_idx + num_samples
    if next_clicks > 0:
        start_idx = min(start_idx + num_samples, total_samples - num_samples)
        end_idx = start_idx + num_samples

    # Adjust end_idx if it's beyond the total number of samples
    end_idx = min(end_idx, total_samples)

    # Slice the dataframe
    df_slice = df.iloc[start_idx:end_idx]

    # Update graph
    if col_chosen == 'All':
        fig = go.Figure()
        for axis in ['X-axis (xx)', 'Y-axis (yy)', 'Z-axis (zz)']:
            fig.add_trace(go.Scatter(x=df_slice['Timestamp'], y=df_slice[axis], mode='lines', name=axis))
        fig.update_layout(title='Gyroscope Readings over Timestamp', xaxis_title='Timestamp', yaxis_title='Value')
    else:
        if graph_type == 'line':
            fig = px.line(df_slice, x='Timestamp', y=col_chosen, title=f'{col_chosen} over Timestamp')
        elif graph_type == 'scatter':
            fig = px.scatter(df_slice, x='Timestamp', y=col_chosen, title=f'{col_chosen} over Timestamp')
        elif graph_type == 'distribution':
            fig = px.histogram(df_slice, x=col_chosen, title=f'Distribution of {col_chosen}')
        
        fig.update_layout(xaxis_title='Timestamp' if graph_type in ['line', 'scatter'] else col_chosen,
                          yaxis_title=col_chosen)
    
    # Calculate statistics
    stats = df_slice.describe().transpose().reset_index()
    stats = stats.rename(columns={'index': 'Metric'})
    summary_data = stats.to_dict('records')

    return fig, summary_data

if __name__ == '__main__':
    app.run_server(mode='inline', port=8060)



DataFrame columns: Index(['Timestamp', 'X-axis (xx)', 'Y-axis (yy)', 'Z-axis (zz)'], dtype='object')
